In [1]:
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Conv2D, MaxPooling2D
from tensorflow.python.keras.layers import Activation, Dropout, Flatten, Dense

In [2]:
# Каталог с данными для обучения
train_dir = './data2.0/train'
# Каталог с данными для проверки
val_dir = './data2.0/val'
# Каталог с данными для тестирования
test_dir = './data2.0/test'
# Размеры изображения
img_width, img_height = 16, 16
# Размерность тензора на основе изображения для входных данных в нейронную сеть
# backend Tensorflow, channels_last
input_shape = (img_width, img_height, 3)
# Количество эпох
epochs = 5

In [3]:
datagen = ImageDataGenerator(rescale=1. / 255)

Генератор данных для обучения на основе изображений из каталога

In [4]:
train_generator = datagen.flow_from_directory(
    './data2.0/train',
    target_size=(16, 16),
    batch_size=20,
    class_mode='categorical')

Found 3966 images belonging to 10 classes.


Генератор данных для проверки на основе изображений из каталога

In [5]:
val_generator = datagen.flow_from_directory(
    './data2.0/val',
    target_size=(img_width, img_height),
    batch_size=30,
    class_mode='categorical')


Found 1210 images belonging to 10 classes.


Генератор данных для тестирования на основе изображений из каталога

In [12]:
test_generator = datagen.flow_from_directory(
    './data2.0/test/',
    target_size=(img_width, img_height),
    batch_size=20,
    class_mode='categorical')

Found 1211 images belonging to 10 classes.


# Сверточная нейронная сеть




In [7]:
model = Sequential()

model.add(Conv2D(75, kernel_size=(5, 5),
                 activation='relu',
                 input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))
model.add(Conv2D(100, (5, 5), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(500, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

In [8]:
# Компилируем модель
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

In [9]:
model.fit_generator(
    train_generator,
    steps_per_epoch=200,
    epochs=epochs,
    validation_data=val_generator,
    validation_steps=60)

Epoch 1/5
200/200 [==============================]200/200 [==============================] - 590s 3s/step - loss: 1.4712 - acc: 0.4424 - val_loss: 0.4617 - val_acc: 0.8264

Epoch 2/5
200/200 [==============================]200/200 [==============================] - 585s 3s/step - loss: 0.3233 - acc: 0.8830 - val_loss: 0.3464 - val_acc: 0.8955

Epoch 3/5
200/200 [==============================]200/200 [==============================] - 50902s 255s/step - loss: 0.1925 - acc: 0.9232 - val_loss: 0.2161 - val_acc: 0.9256

Epoch 4/5
200/200 [==============================]200/200 [==============================] - 649s 3s/step - loss: 0.1387 - acc: 0.9447 - val_loss: 0.1975 - val_acc: 0.9348

Epoch 5/5
200/200 [==============================]200/200 [==============================] - 13041s 65s/step - loss: 0.1006 - acc: 0.9662 - val_loss: 0.1527 - val_acc: 0.9517



Сохранение модели

In [10]:
model_json = model.to_json()
# Записываем модель в файл
json_file = open("model.json", "w")
json_file.write(model_json)
json_file.close()

In [11]:
#Сохраняем веса модели
model.save_weights("model_weights.h5")

## Загружаем модель 

In [86]:
import numpy as np
from keras.preprocessing import image
from PIL import Image
from keras.utils import np_utils
from keras.models import model_from_json
import os

In [80]:
img_path = './data2.0/test2/3/pixil-frame-0 (1).png'
img = image.load_img(img_path, target_size=(16, 16))
x = image.img_to_array(img)
x /= 255
x = np.expand_dims(x, axis=0)

json_file = open('model.json', 'r')

loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)

loaded_model.load_weights("model_weights.h5")

loaded_model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
prediction = loaded_model.predict(x)
prediction = np.argmax(prediction, axis=1)
print(prediction)

[3]


## Тестируем модель

In [85]:
scores = model.evaluate_generator(test_generator, 60)
print("Точность на тестовых данных: %.2f%%" % (scores[1]*100))

Точность на тестовых данных: 94.92%


In [124]:
path = './data2.0/test2/10/'
predictions=[]
print()
for image_path in sorted(os.listdir(path)):
    img = image.load_img(path + image_path, target_size=(16, 16))
    x = image.img_to_array(img)
    x /= 255
    x = np.expand_dims(x, axis=0)

    prediction = model.predict(x)
    
    
    prediction = np.argmax(prediction, axis=1)[0]
    
    if prediction == 1:
        prediction = 10
    elif prediction == 0:
        prediction = 1
        
    predictions.append(prediction)
    print('{0:^12} -- это цифра {1}'.format(image_path,prediction))



  10,1.png   -- это цифра 5
 10,10.png   -- это цифра 10
  10,2.png   -- это цифра 5
  10,3.png   -- это цифра 10
  10,4.png   -- это цифра 10
   10.png    -- это цифра 10
pixil-frame-0 (1).png -- это цифра 10
pixil-frame-0 (2).png -- это цифра 10
pixil-frame-0.png -- это цифра 10


In [117]:
test2_generator = datagen.flow_from_directory(
    './data2.0/test2/',
    target_size=(img_width, img_height),
    batch_size=20,
    class_mode='categorical')

Found 91 images belonging to 10 classes.


In [89]:
scores = model.evaluate_generator(test2_generator, 4)
scores

[1.5062660574913025, 0.72499999403953552]